<a href="https://colab.research.google.com/github/wuxun90/jiu_cai/blob/master/jiucai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install yfinance plotly==4.6.0 bs4 progressbar2


     |████████████████████████████████| 7.2MB 2.3MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [0]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
import requests
import bs4 as bs
import progressbar
import math

def fetch_sp500_tickers():
    resp = requests.get(
        'http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.content, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker.rstrip())
    return tickers

def get_gain_optimistic(prices, index, max_interval, stop_percentage):
    if index + max_interval + 1 >= len(prices['Open']):
        return 0
    cost = max(prices['Open'][index], 0.001)
    sell_price = 0
    low_stop_sell = 0
    high_stop_sell = 0
    for i in range(index, index + max_interval + 1):
        if (prices['High'][i] - cost) / cost >= stop_percentage:
            return stop_percentage
        if (cost - prices['Low'][i]) / cost >= stop_percentage:
            return -stop_percentage
    return (prices['Open'][index + max_interval + 1] - cost) / cost

def get_gain_pessimistic(prices, index, max_interval, stop_percentage):
  if index + max_interval + 1 >= len(prices['Open']):
    return 0
  cost = max(prices['Open'][index], 0.001)
  sell_price = 0
  low_stop_sell = 0
  high_stop_sell = 0
  for i in range(index, index + max_interval + 1):
    if (cost - prices['Low'][i]) / cost >= stop_percentage:
        return -stop_percentage
    if (prices['High'][i] - cost) / cost >= stop_percentage:
        return stop_percentage
  return (prices['Open'][index + max_interval + 1] - cost) / cost

def get_trend(prices, start_index, end_index):
  ascending = True
  descending = True
  for i in range(start_index, end_index + 1):
    if prices['Open'][i] < prices['Close'][i]:
        descending = False
    elif prices['Open'][i] > prices['Close'][i]:
        ascending = False
  if ascending:
    return 1
  if descending:
    return -1
  return 0

def is_down_shooting_star(prices, index, threshold):
  upper_bound = max(prices['Open'][index], prices['Close'][index])
  lower_bound = min(prices['Open'][index], prices['Close'][index])
  return ((index > 4) and
          ((prices['High'][index] - upper_bound) / max(upper_bound - lower_bound, 0.001) >= threshold) and 
          ((prices['High'][index] - upper_bound) / max(lower_bound - prices['Low'][index], 0.001) >= threshold) and 
          (get_trend(prices, index - 4, index - 1) == 1))

def is_up_shooting_star(prices, index, threshold):
  upper_bound = max(prices['Open'][index], prices['Close'][index])
  lower_bound = min(prices['Open'][index], prices['Close'][index])
  return ((index > 4) and
          ((lower_bound - prices['Low'][index]) / max(upper_bound - lower_bound, 0.001) >= threshold) and 
          ((lower_bound - prices['Low'][index]) / max(prices['High'][index] - upper_bound, 0.001) >= threshold) and 
          (get_trend(prices, index - 4, index - 1) == -1))

def is_down_engulfing(prices, index, threshold):
  return ((index > 4) and
          (prices['Open'][index] > prices['Close'][index]) and
          (prices['Close'][index-1] > prices['Open'][index-1]) and
          (prices['Open'][index] > prices['Close'][index-1]) and
          (prices['Close'][index] < prices['Open'][index-1]) and
          ((prices['Open'][index] - prices['Close'][index]) / max(prices['Close'][index] - prices['Open'][index], 0.001) >= threshold) and
          (get_trend(prices, index - 4, index - 1) == 1))
  
def is_up_engulfing(prices, index, threshold):
  return ((index > 4) and
          (prices['Open'][index] < prices['Close'][index]) and
          (prices['Close'][index-1] < prices['Open'][index-1]) and
          (prices['Close'][index] > prices['Open'][index-1]) and
          (prices['Open'][index] < prices['Close'][index-1]) and
          ((prices['Close'][index] - prices['Open'][index]) / max(prices['Open'][index] - prices['Close'][index], 0.001) >= threshold) and
          (get_trend(prices, index - 4, index - 1) == -1))




In [33]:
tickers = fetch_sp500_tickers()
down_shooting_star_ticker_list = {}
up_shooting_star_ticker_list = {}
down_engulfing_ticker_list = {}
up_engulfing_ticker_list = {}

with progressbar.ProgressBar(max_value=len(tickers)) as bar:
  i = 0
  for ticker in tickers:
    i += 1
    bar.update(i)
    down_shooting_stars = []
    up_shooting_stars = []
    down_engulfings = []
    up_engulfings = []
    price = yf.Ticker(ticker)
    period = '60d' # m = min, h = hour, d = day, wk = week, mo=month
    interval = '15m' # valid intervals 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
    prices = price.history(period=period, interval=interval)
    for index in range(prices.shape[0]):
      if is_down_shooting_star(prices, index, 3) == True:
        # Since we are shorting stock, we need to be "optimistic" for the worst case.
        down_shooting_stars.append((index, get_gain_optimistic(prices, index + 1, 4, 0.05)))
      if is_up_shooting_star(prices, index, 3) == True:
        # Since we are longing stock, we need to be "pessimistic" for the worst case.
        up_shooting_stars.append((index, get_gain_pessimistic(prices, index + 1, 5, 0.05)))
      if is_down_engulfing(prices, index, 3) == True:
        # Since we are shorting stock, we need to be "optimistic" for the worst case.
        down_engulfings.append((index, get_gain_optimistic(prices, index + 1, 4, 0.05)))
      if is_up_engulfing(prices, index, 3) == True:
        # Since we are longing stock, we need to be "pessimistic" for the worst case.
        up_engulfings.append((index, get_gain_pessimistic(prices, index + 1, 3, 0.05)))
    if down_shooting_stars:
      down_shooting_star_ticker_list[ticker] = down_shooting_stars
    if up_shooting_stars:
      up_shooting_star_ticker_list[ticker] = up_shooting_stars
    if down_engulfings:
      down_engulfing_ticker_list[ticker] = down_engulfings
    if up_engulfings:
      up_engulfing_ticker_list[ticker] = up_engulfings

N/A% (0 of 505) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

Start loading tickers...


 13% (69 of 505) |###                    | Elapsed Time: 0:01:04 ETA:   0:07:55

- BRK.B: No data found, symbol may be delisted


 16% (82 of 505) |###                    | Elapsed Time: 0:01:15 ETA:   0:04:51

- BF.B: No data found for this date range, symbol may be delisted


100% (505 of 505) |######################| Elapsed Time: 0:07:53 Time:  0:07:53


In [34]:
for ticker_list_tuple in [('down_shooting_star', down_shooting_star_ticker_list),
                          ('up_shooting_star', up_shooting_star_ticker_list),
                          ('down_engulfing', down_engulfing_ticker_list),
                          ('up_engulfing', up_engulfing_ticker_list)]:
  total_gain = 0
  positive = 0
  negative = 0
  for _, ticker_earnings in ticker_list_tuple[1].items():
      for pair in ticker_earnings:
          gain = pair[1]
          if math.isnan(gain):
              gain = 0
          total_gain += gain
          if gain > 0:
              positive += 1
          else:
              negative += 1
  print("Total gain for %s: %s; Positive number: %d; Negative number: %d" % (ticker_list_tuple[0], total_gain, positive, negative))

Total gain for down_shooting_star: -2.453281103363123; Positive number: 1142; Negative number: 1252
Total gain for up_shooting_star: 3.145935927997773; Positive number: 1466; Negative number: 1216
Total gain for down_engulfing: -5.547895398260688; Positive number: 1731; Negative number: 1944
Total gain for up_engulfing: -0.5688594648645952; Positive number: 1918; Negative number: 1967


In [35]:
up_engulfing_ticker_list['GOOG']

[(55, 0.0023668952020785886),
 (131, -0.0006015824233308346),
 (341, 0.016638276253660857),
 (460, -0.0017309750110153313),
 (745, -0.005642089019626667),
 (771, 0.013581206937085177),
 (799, -0.018922018348623695),
 (805, 0.011298977968341232),
 (1041, 0.008316095371293195),
 (1188, 0.00258410336413458),
 (1308, 0.0022530775167810403),
 (1317, -0.0006895957245064155)]

In [36]:
ticker_name = 'GOOG'
record_list = up_engulfing_ticker_list

ticker_fetcher = yf.Ticker(ticker_name)
ticker_prices = ticker_fetcher.history(period=period, interval=interval)
ticker_points = np.array([item[0] for item in record_list[ticker_name]])
plot_list = np.concatenate((ticker_points, ticker_points-1, ticker_points-2,
                            ticker_points-3,ticker_points+1, ticker_points+2,
                            ticker_points+3,ticker_points+4, ticker_points+5))
plot_list = np.sort(plot_list)

fig = go.Figure(data=[go.Candlestick(x=ticker_prices.index[plot_list],
                                     open=ticker_prices['Open'][plot_list], 
                                     high=ticker_prices['High'][plot_list],
                                     low=ticker_prices['Low'][plot_list],
                                     close=ticker_prices['Close'][plot_list])])
fig.show()